In [ ]:

import math, json, re
import ccxt
wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['last']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['last'])})

print(data_for_graph)


Создаем списки смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид.

In [10]:
graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = -math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print(graph)
# воплощение алгоритма отсюда
#  https://gist.github.com/joninvski/701720


{b'NMCET': {b'NMC': 0.3915622029391729}, b'ETHET': {b'ETH': 0.507497833673316}, b'NMC': {b'BTC': 8.517193191416238, b'NMCET': -0.391562202939173, b'USD': -0.5596157879354227}, b'BTC': {b'ZEC': -3.575550768806933, b'DASH': -3.024955132230894, b'NMC': -8.517193191416238, b'ETH': -2.7954070923204615, b'EUR': -8.853106403240773, b'USD': -9.057118094383503, b'LTC': -3.979231755121596, b'RUR': -13.085813331894157, b'NVC': -7.706262975199909, b'PPC': -8.377431249041079, b'BTCET': -0.41248972304512876, b'BCH': -2.1698039817602273}, b'EUR': {b'LTC': 4.873845264543479, b'EURET': -0.44316697529217586, b'DASH': 5.829660068211937, b'BCH': 6.681948183581686, b'BTC': 8.853106403240773, b'ETH': 6.060276922306459, b'RUR': -4.236070661334393, b'USD': -0.203161227873645}, b'DSHET': {b'DASH': 0.33687231664255274}, b'BCHET': {b'BCH': 0.38272562113867487}, b'ZEC': {b'LTC': -0.4027948795522855, b'USD': -5.480380556634855, b'BCH': 1.404870746692826, b'BTC': 3.575550768806933, b'ETH': 0.7770286645406473, b'DAS

In [ ]:

# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [16]:
 paths = []
 for key in graph:

    path = bellman_ford(graph, key)

    if path not in paths and not None:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(-graph[start][end])
                money *= rate
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

Starting with 100 in b'NVC'
b'NVC' to b'USD' at 3.854000 = 385.400000
b'USD' to b'NVC' at 0.259471 = 100.000000
Starting with 100 in b'USD'
b'USD' to b'NMC' at 0.571429 = 57.142857
b'NMC' to b'BTC' at 0.000200 = 0.011429
b'BTC' to b'NVC' at 2222.222222 = 25.396825
b'NVC' to b'USD' at 3.854000 = 97.879365
Starting with 100 in b'BTC'
b'BTC' to b'NVC' at 2222.222222 = 222222.222222
b'NVC' to b'USD' at 3.854000 = 856444.444444
b'USD' to b'NMC' at 0.571429 = 489396.825397
b'NMC' to b'BTC' at 0.000200 = 97.879365
Starting with 100 in b'USD'
b'USD' to b'NVC' at 0.259471 = 25.947068
b'NVC' to b'USD' at 3.854000 = 100.000000


